In [ ]:
# Writed in August 31, 2023
# Author : Yukui Min
# Email : 2210902155@cnu.edu.cn
# Institude :Capital Normal University, Beijing, 100048, China

In [ ]:
import ee
import collections
import os
import geemap
import subprocess
import math
import helper_function as F
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:33210'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:33210'
collections.Callable = collections.abc.Callable
ee.Initialize()

In [62]:
# load ROI data
ROI = 'D:/hhmc_roi/hhmc2/HHMC_SD/HHMC_SD_p1.shp'
ROI = geemap.shp_to_ee(ROI)
ROI = ROI.geometry()
Map = geemap.Map()
Map.addLayer(ROI)

tide_roi1 = ROI
# Map

In [14]:

# tide_roi1 = Map.draw_last_feature
# tide_roi1 = tide_roi1.geometry()

In [40]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
#L8  = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

bandIn_S2 = ['B2','B3','B4','B8','cloud']
bandOut = ['blue','green','red','nir','cloud']
#bandIn_L8 = ['SR_B2','SR_B3','SR_B4','SR_B5','QA_PIXEL']
bandIn_L8 = ['B2','B3','B4','B5','pixel_qa']


start_date = '2021-01-01'
end_date = '2021-12-17'
MGRS_TILE = '50SPG'
SENSING_ORBIT_NUMBER = 132
CLOUDY_PIXEL_PERCENTAGE = 30
WRS_PATH = 121
WRS_ROW = 34

In [41]:
# Sentinel-2 cloud removal by S2cloudless product
S2_cloud_removal = F.get_s2_sr_cld_col(tide_roi1, start_date, end_date,MGRS_TILE,SENSING_ORBIT_NUMBER,CLOUDY_PIXEL_PERCENTAGE)

# add cloud band
S2_cloudless = S2_cloud_removal.map(F.add_cloud_bands)

# Bands rename
S2_datesat_2021 = S2_cloudless.select(bandIn_S2,bandOut)

In [43]:

def CLIP(img):
    return img.clip(tide_roi1).unmask(1)

# import Sentinel-2 SR and Landsat 8 SR (Surface reflectance)

S2_datesat_2021 = S2_datesat_2021.filterBounds(tide_roi1)\
            .filterDate(start_date,end_date)\
            .filter(ee.Filter.eq('MGRS_TILE',MGRS_TILE))\
            .filter(ee.Filter.eq('SENSING_ORBIT_NUMBER',SENSING_ORBIT_NUMBER))\
            .map(F.addtime)\
            .map(CLIP).filter(ee.Filter.neq('DOY',326)) # Remove poor quality images

L8_datesat_2021 = L8.filterBounds(tide_roi1)\
            .filterDate(start_date,end_date)\
            .filter(ee.Filter.lt('CLOUD_COVER',CLOUDY_PIXEL_PERCENTAGE))\
            .filter(ee.Filter.eq('WRS_PATH',WRS_PATH))\
            .filter(ee.Filter.eq('WRS_ROW',WRS_ROW))\
            .select(bandIn_L8,bandOut)\
            .map(F.cloud_mask_L8)\
            .map(F.addtime)\
            .map(CLIP)\
            .filter(ee.Filter.neq('DOY',156))\
            .filter(ee.Filter.neq('DOY',316))\
            .filter(ee.Filter.neq('DOY',348)) # Remove poor quality images

In [49]:
# Sentinel-2 and Landsat 8 fusion
# Step 1 : Spatial register and resample 
# Step 2 : Band adjust by transformation functions developed by Xie et al. (2022)
# citation ：Xie, S., Sun, L., Liu, L., Liu, X., 2022. Global Cross-Sensor Transformation Functions for Landsat-8 and Sentinel-2 Top of 
# Atmosphere and Surface Reflectance Products Within Google Earth Engine. IEEE Trans. Geosci. Remote Sens., 60, 1-9.

refer_img = S2_datesat_2021.first()

# transformation parameters 
SlopeL8 = [1.0296,1.0661,0.9804,1.0326,1,1]
offsetL8 = [17,-13,39,101,0,0]

S2_datesat_2021 =  S2_datesat_2021.map(F.toint)
S2_LIST = S2_datesat_2021.toList(S2_datesat_2021.size())
mlist = S2_LIST
L8_LIST = L8_datesat_2021.toList(L8_datesat_2021.size())
n = L8_datesat_2021.size().getInfo()

for i in range(n):
    
    img = ee.Image(L8_LIST.get(i))
    # Spatial_register ( Landsat 8 to Sentinel-2 )
    registered = F.Spatial_register(img,refer_img) 
    
    # Landsat 8 resample to 10m by bicubic 
    resampled = ee.Image(F.resample_L8(registered,refer_img))
    
    # band adjustment by transformation functions developed by Xie et al. (2022)
    harmoic_L8 = F.band_adjust(resampled,SlopeL8,offsetL8).multiply(1000).toInt().divide(1000)#.unmask(1)
    
    mlist = mlist.add(harmoic_L8)
    
collect = ee.ImageCollection.fromImages(mlist)

# fusion collection sort by images time (DOY)

doy_array = collect.toArray().arraySlice(1,5,6)
sorted_array =  collect.toArray().arraySort(doy_array)

n = collect.size().getInfo()
name = [str(j) for j in range(n)]

bandname = ['blue','green','red','nir','cloud','doy']
img_list = ee.List([])
for i in range(n):
    img = sorted_array.arraySlice(0,i,i+1).arrayProject([1]).arrayFlatten([bandname])
    img_list = img_list.add(img)
    
datesat_2021 = ee.ImageCollection.fromImages(img_list)


9


In [51]:

# Filling cloud-pixels by time-series linear interpolation 

img_list_2021 = datesat_2021.toList(datesat_2021.size())
n = datesat_2021.size().getInfo()
datesat_QY_2021 = ee.ImageCollection(ee.Image(img_list_2021.get(0)).multiply(1000).toInt().divide(1000))

for k in range(1,n-1):
    img = ee.Image(img_list_2021.get(k))
    img_before = ee.Image(img_list_2021.get(k-1))
    img_after = ee.Image(img_list_2021.get(k+1))
    image = ee.ImageCollection(F.linear_interpolate(img,img_before,img_after))
    datesat_QY_2021 = datesat_QY_2021.merge(image)

datesat_QY_2021 = datesat_QY_2021.merge(ee.ImageCollection(ee.Image(img_list_2021.get(n-1)).multiply(1000).toInt().divide(1000)))
datesat_QY_2021 = datesat_QY_2021.map(F.addNDVI).map(F.addNDWI)
datesat_QY_2021 = datesat_QY_2021.select(['doy','NDVI','NDWI'])

# unmask Null pixels
def bc(img):
    return img.unmask(0).clip(tide_roi1)

datesat_QY_2021 = datesat_QY_2021.map(bc)

In [52]:
# define bands name
n = datesat_QY_2021.size().getInfo()
NDVI_no = [k for k in range(0,n)]
NDVI_no = ['NDVI'+str(k) for k in NDVI_no]
# print(NDVI_no)

['NDVI0', 'NDVI1', 'NDVI2', 'NDVI3', 'NDVI4', 'NDVI5', 'NDVI6', 'NDVI7', 'NDVI8', 'NDVI9', 'NDVI10', 'NDVI11', 'NDVI12', 'NDVI13', 'NDVI14', 'NDVI15', 'NDVI16', 'NDVI17', 'NDVI18', 'NDVI19', 'NDVI20', 'NDVI21', 'NDVI22', 'NDVI23', 'NDVI24', 'NDVI25', 'NDVI26', 'NDVI27', 'NDVI28', 'NDVI29', 'NDVI30', 'NDVI31', 'NDVI32', 'NDVI33', 'NDVI34', 'NDVI35', 'NDVI36', 'NDVI37', 'NDVI38', 'NDVI39', 'NDVI40', 'NDVI41', 'NDVI42']


In [53]:
# Apply Tide Gap Filling aglorithm in NDVI timeseries
Bands = 'NDVI'
datesat_QY_TGF_2021 = F.apply_TGF_to_timeseries(datesat_QY_2021,Bands)

In [55]:
# get NDVI-loacl-maximum (NDVIlm) time series
composite_datesat_2021 = F.NDVI_local_maximum(datesat_QY_TGF_2021)

# count  differences of each pair of adjacent observations (∆𝑁𝐷𝑉𝐼) 
dif_NDVI_composite_datesat_2021 = F.count_VI_dif(composite_datesat_2021)

#
filter_datesat_s_2021 = F.NDVI_lm_sort(datesat_QY_TGF_2021,dif_NDVI_composite_datesat_2021)

# Extract Potential removal peorid ---> PCT_datesat_2021
# PCT_datesat_2021 is ImageCollection include six images with bands of doy and NDVI
PCT_datesat_2021 = F.extract_potential_removal_period(filter_datesat_s_2021)

15


In [56]:
# dif_NDVI_composite_datesat is  DIF_NDVI

DIF_NDVI_max = dif_NDVI_composite_datesat_2021.qualityMosaic('NDVI')
t_before = DIF_NDVI_max.select('doy_before') # t1
t_after =  DIF_NDVI_max.select('doy_after')  # t2

# S. alterniflora standard phenological curves of Shandong
a0 = 0.34824
a1 = -0.06868
b1 = -0.22907
a2 = -0.01245
b2 = -0.0168

#y = a0 + a1*cos(2*pi*t/365) + b1*sin(2*pi*t/365) + a2*cos(4*pi*t/365) + b2*sin(4*pi*t/365)

y1 = ((t_before.multiply(2*math.pi/365)).cos().multiply(a1)).add((t_before.multiply(2*math.pi/365)).sin().multiply(b1))
y2 = ((t_before.multiply(4*math.pi/365)).cos().multiply(a2)).add((t_before.multiply(4*math.pi/365)).sin().multiply(b2))
y_before = (y1.add(y2).add(a0)).multiply(1000).toInt().divide(1000)

y3 = ((t_after.multiply(2*math.pi/365)).cos().multiply(a1)).add((t_after.multiply(2*math.pi/365)).sin().multiply(b1))
y4 = ((t_after.multiply(4*math.pi/365)).cos().multiply(a2)).add((t_after.multiply(4*math.pi/365)).sin().multiply(b2))
y_after = (y3.add(y4).add(a0)).multiply(1000).toInt().divide(1000)



# PCT_dif_NDVI ---> ∆𝑁𝐷𝑉𝐼 
# Dy_thred -->  ∆𝑁𝐷𝑉𝐼pheno
PCT_dif_NDVI = dif_NDVI_composite_datesat_2021.qualityMosaic('NDVI').select('NDVI')
Dy_thred = (y_before.subtract(y_after)).rename('NDVI')
dif_Dy_thred = PCT_dif_NDVI.subtract(Dy_thred)

#   get_last_img_NDVI  ( NDVI_last )

n = datesat_QY_TGF_2021.size().getInfo()
datesat_list = datesat_QY_TGF_2021.toList(n)
img = ee.Image(datesat_list.get(n-1))
img_NDVI = img.select('NDVI')

In [57]:
Non_clearing_mask_2021  = (img_NDVI.gt(0.32).And(dif_Dy_thred.lte(0.1))).Or(img_NDVI.gt(0.45))

viz1 = {
    'bands':['doy_after'],
    'max':365,
    'min':150,
    'palette': ['276419','d7301f','ef6548','fc8d59','fdbb84','fdd49e','fee8c8','f7fcf0','e0f3db','ccebc5','a8ddb5','7bccc4','4eb3d3','2b8cbe','0868ac']
}

PCT_datesat_dif_2021 = F.count_VI_dif(PCT_datesat_2021)
MM_2021 = PCT_datesat_dif_2021.qualityMosaic('NDVI')

clearing_mask_2021 = Non_clearing_mask_2021.add(-1)
clearing_mask_2021 = clearing_mask_2021.where(clearing_mask_2021.lt(0),1)


clearing_region_2021 = MM_2021.updateMask(clearing_mask_2021)
Non_clearing_region_2021 = (MM_2021.updateMask(Non_clearing_mask_2021)).multiply(0)
merge_2021 = ee.ImageCollection([clearing_region_2021,Non_clearing_region_2021])

hhmc_shp = HHMC4_DZG
harvest_date_2021 = merge_2021.mosaic()

6


In [58]:
hhmc_shp = HHMC4_DZG
harvest_date_2021 = merge_2021.mosaic()
harvest_date_2021_clip = harvest_date_2021.clip(hhmc_shp)

In [28]:
QQ = geemap.Map()
QQ.addLayer(harvest_date_2021_clip,viz1,'harvest_date_2022_clip')

In [61]:
# Export 

root_file = 'D:/hhmc_roi/hhmc2/roi_xz/'
out_dir = 'D:\hhmc_roi\SD_result\zasfc'
for i in range(1,10):
    roi_folder = root_file + 'HHK'+str(i)+'.shp'
    #print(roi_folder)
    roi = geemap.shp_to_ee(roi_folder)
    roi = roi.geometry()
    
    name = 'S2_HHKb_kc' + str(i) + '.tif'
    filename = os.path.join(out_dir, name)
    print(filename)
    harvest_date_2021_k = harvest_date_2021.select('doy_after')
    geemap.ee_export_image(
        harvest_date_2021_k, filename=filename, scale=10, region=roi, file_per_band=False
    )
